In [15]:
import glob
from collections import defaultdict
from pprint import pprint
import pickle
import operator
import os
import nltk
import sklearn
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.util import ngrams
import random
import re
from sklearn.model_selection import GridSearchCV
from IPython.core.display import display, HTML

In [7]:
orig_paper_predicate_expr = set(["great", "so much fun", "good", "so happy", "better", "my favorite thing", "cool", "funny", "nice", "always fun", "fun", "awesome", "the best feeling", "amazing", "happy", "ready today", "ready", "dry", "juicy", "my favorite part"])
orig_paper_pos_expr = set(["love", "missed", "loves", "enjoy", "cant wait", "excited", "wanted", "can't wait", "get", "appreciate", "decided", "loving", "really like", "looooove", "just keeps", "loveee", "randomly stop", "cannot wait", "just live", "please keep", "live", "stoked", "goin", "reading", "break", "just stops", "stops"])
orig_paper_neg_expr = set(["being ignored", "being sick", "waiting", "feeling", "waking up early", "being woken", "fighting", "staying", "writing", "being home", "cleaning", "not getting", "crying", "sitting at home", "being stuck", "starting", "being told", "being left", "getting ignored", "being treated", "doing homework", "learning", "getting up early", "going to bed", "getting sick", "riding", "being ditched", "getting ditched", "missing", "not sleeping", "not talking", "trying", "falling", "walking home", "getting yelled", "being awake", "being talked", "taking care", "doing nothing", "wasting", "throwing", "getting woken up", "to spend", "standing", "smelling", "getting woken", "arguing", "paying bills", "being locked", "shoveling", "getting called", "being at work", "having nothing", "getting invited", "getting blown", "dealing", "ending", "to wake", "when doesn't text", "getting ready", "to learn", "picking", "walking to class", "breaking", "being invited", "getting home", "setting", "dropping", "not seeing", "forgetting", "being called fat", "getting lied", "invited", "to sit here", "to be ignored", "being late", "doing laundry", "being taken", "practicing", "babysitting", "getting hit", "being used", "being used", "being reminded", "when falls", "working all day", "running late", "traveling", "peeing", "being hit", "having practice", "not being invited", "being bored", "stepping", "spending my day", "leaving", "almost getting", "being put", "passing", "being at school", "to study", "going to class", "coughing", "sitting in traffic", "being yelled", "fixing", "burning", "walking to school", "wakin", "seeing people", "being accused", "being up early", "scratches", "texting someone", "being invited places", "receiving", "being grounded", "checking", "getting my ass", "getting back", "getting bitched", "getting treated", "only getting", "reviewing", "sitting alone", "getting screwed", "going there", "getting stared", "calling", "watching scary movies", "getting no sleep", "taking tests", "getting locked", "reading tweets", "teaching", "waking up not", "sounding", "getting made", "sleeping alone", "not feeling", "being surrounded", "editing", "being stood up", "to randomly ask", "getting hacked", "getting texts", "having insomnia", "having homework", "blamed", "showing", "being blamed", "getting bad news", "getting played", "being stood", "scrolling", "being lied too", "being a loner", "going weeks", "being up late", "having class", "failing", "being cussed", "listening to women", "when ignores", "cutting", "bring", "burnt", "getting hate", "coming to school", "sitting here", "waking up early", "being called names", "getting replaced", "having bruises", "closing", "coming back", "getting punched", "getting phone", "spending all day", "being pushed", "spending", "not being able", "waking", "working", "sitting", "walking", "coming home", "living", "being lied", "getting", "coming", "going", "running", "to sit", "being called", "to read", "studying", "paying", "texting", "hearing", "replying", "gettin better", "gettin better", "gettin", "eating", "losing", "listening", "to get up", "finding", "to clean", "being able", "seeing", "to run", "to drive", "to go back", "looking", "taking", "putting", "driving", "to start", "posting", "to pay", "telling me", "ruined", "being woke", "hitting", "laying", "cuddling", "reading", "buying", "cancelled", "sending", "to see pictures", "to find out", "sharing", "finishing", "sweating", "to miss", "hurting"])

In [8]:
x_train,y_train,x_test,y_test = pickle.load(open('march27/dataset_used-includes_train_test_split.pkl','rb'))

In [11]:
categories = ['NOT_SARCASM','SARCASM']

In [22]:
def allindices(string, sub):
    l=[]
    i = string.find(sub)
    while i >= 0:
        l.append(i)
        i = string.find(sub, i + 1)
    return l

In [24]:
predicted = []
display(HTML('<b>Positive VPs</b>'))
for tweet in x_test:
    predicted.append('NOT_SARCASM')
    for posvp in orig_paper_pos_expr:
        if posvp in tweet:
            predicted[-1] = "SARCASM"
            break
print(metrics.classification_report(y_test,predicted,target_names=categories)) 

predicted = []
display(HTML('<b>Negative Situations</b>'))
for tweet in x_test:
    predicted.append('NOT_SARCASM')
    for negsit in orig_paper_neg_expr:
        if negsit in tweet:
            predicted[-1] = "SARCASM"
            break
print(metrics.classification_report(y_test,predicted,target_names=categories)) 

predicted = []
display(HTML('<b>Contrast(+VPs, –Situations), Unordered</b>'))
for tweet in x_test:
    predicted.append('NOT_SARCASM')
    for negsit in orig_paper_neg_expr:
        break_negsit = False
        for posvp in orig_paper_pos_expr:
            if negsit in tweet and posvp in tweet:
                predicted[-1] = "SARCASM"
                break_negsit == True
                break
        if break_negsit:
            break
print(metrics.classification_report(y_test,predicted,target_names=categories)) 

predicted = []
display(HTML('<b>Contrast(+VPs, –Situations), Ordered</b>'))
for tweet in x_test:
    predicted.append('NOT_SARCASM')
    for posvp in orig_paper_pos_expr:
        indices = allindices(tweet,posvp)
        break_posvp = False
        if indices:
            for negsit in orig_paper_neg_expr:
                for idx in indices:
                    if negsit in tweet[idx+len(posvp):]:
                        predicted[-1] = "SARCASM"
                        break_posvp == True
                        break
        if break_posvp:
            break
print(metrics.classification_report(y_test,predicted,target_names=categories)) 

             precision    recall  f1-score   support

NOT_SARCASM       0.84      0.82      0.83      1512
    SARCASM       0.40      0.44      0.42       412

avg / total       0.75      0.74      0.74      1924



             precision    recall  f1-score   support

NOT_SARCASM       0.82      0.85      0.84      1512
    SARCASM       0.37      0.31      0.33       412

avg / total       0.72      0.74      0.73      1924



             precision    recall  f1-score   support

NOT_SARCASM       0.81      0.94      0.87      1512
    SARCASM       0.46      0.20      0.28       412

avg / total       0.74      0.78      0.74      1924



             precision    recall  f1-score   support

NOT_SARCASM       0.80      0.98      0.88      1512
    SARCASM       0.60      0.12      0.21       412

avg / total       0.76      0.79      0.74      1924



In [4]:
results = pickle.load(open('results_summary.pkl','rb'))

In [31]:
riloff_eval_dataset = pickle.load(open('data/riloff-emnlp/sarcasm-annos-emnlp13-tweet_objs-2124.pkl','rb'))

In [32]:
riloff_eval_dataset[0]['text']

'Very low entries in this giveaway! Hop over and check it out! http://t.co/OrUSN9ne'

In [35]:
x = []
y = []
with open('data/riloff-emnlp/sarcasm-annos-emnlp13-tweets-noids.tsv') as f:
    for line in f:
        label,tweet = line.strip().split('\t')
        x.append(tweet)
        y.append(label)

In [38]:
z = list(zip(x,y))

In [41]:

for i in range(10):
    random.shuffle(z)

In [42]:
x,y = zip(*z)

In [43]:

c = defaultdict(int)
for label in y:
    c[label] += 1

In [44]:
c

defaultdict(int, {'NOT_SARCASM': 1666, 'SARCASM': 458})

In [3]:
riloff_eval = pickle.load(open('../../riloff-tokenized-and-tagged-lowercase.pkl','rb'))    

In [4]:
len(riloff_eval)

2124

ImportError: No module named 'nltk'

In [13]:

bigrams = ngrams(riloff_eval[0]['tokens'],2)

In [14]:
print(list(bigrams))

[('very', 'low'), ('low', 'entries'), ('entries', 'in'), ('in', 'this'), ('this', 'giveaway'), ('giveaway', '!'), ('!', 'hop'), ('hop', 'over'), ('over', 'and'), ('and', 'check'), ('check', 'it'), ('it', 'out'), ('out', '!'), ('!', 'http://t.co/orusn9ne')]


In [72]:
categories = ['NOT_SARCASM','SARCASM']

In [57]:
train_set_x = [tweet.lower().replace('#sarcasm','').replace('#sarcastic','') for tweet in x[:200]]
train_set_y = y[:200]
test_set_x =  [tweet.lower().replace('#sarcasm','').replace('#sarcastic','') for tweet in x[200:]]
test_set_y = y[200:]

In [47]:


count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_set_x)
print(X_train_counts.shape)


#tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
#X_train_tf = tf_transformer.transform(X_train_counts)
#print(X_train_tf.shape)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)

clf = MultinomialNB().fit(X_train_tfidf, train_set_y)
#To try to predict the outcome on a new document we need to extract the features using almost the same feature extracting chain as before. The difference is that we call transform instead of fit_transform on the transformers, since they have already been fit to the training set:
#docs_new = [tweetobj['tweet'] for tweetobj in riloff_eval]
X_new_counts = count_vect.transform(test_set_x)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

count_vect = CountVectorizer()

predicted = clf.predict(X_new_tfidf)

#for doc, category in zip(docs_new, predicted):
#    print('%r => %s' % (doc, twenty_train.target_names[category]))


(200, 1087)
(200, 1087)


In [48]:

np.mean(predicted == test_set_y) 

0.78534303534303529

In [50]:



text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5, random_state=42)),
])
_ = text_clf.fit(train_set_x, train_set_y)
predicted = text_clf.predict(test_set_x)
np.mean(predicted == test_set_y)            


0.7718295218295218

In [51]:
print(metrics.classification_report(test_set_y, predicted,
    target_names=categories))

             precision    recall  f1-score   support

    SARCASM       0.85      0.86      0.85      1511
NOT_SARCASM       0.47      0.46      0.47       413

avg / total       0.77      0.77      0.77      1924



In [58]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5, random_state=42)),
])
_ = text_clf.fit(train_set_x, train_set_y)
predicted = text_clf.predict(test_set_x)
#print(np.mean(predicted == test_set_y))         
print(metrics.classification_report(test_set_y, predicted,
    target_names=categories))

0.766632016632
             precision    recall  f1-score   support

    SARCASM       0.84      0.86      0.85      1511
NOT_SARCASM       0.45      0.41      0.43       413

avg / total       0.76      0.77      0.76      1924



In [60]:
# Moment of truth:
results = pickle.load(open('results_summary.pkl','rb'))

In [73]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5, random_state=42)),
])
_ = text_clf.fit(train_set_x, train_set_y)
predicted = text_clf.predict(test_set_x)
#print(np.mean(predicted == test_set_y)) 
print("SVM with hinge loss")
print(metrics.classification_report(test_set_y, predicted,
    target_names=categories))

for _id,resultset in results.items():
    print(_id)
    for dataset,wordsets in resultset.items():
        predicted = []
        if dataset == 'shereen': 
            print('DatasetSarc')
        else: 
            print('DatasetOther')
        #print(dataset)
        for tweet in test_set_x:
            predicted.append('NOT_SARCASM')
            for posphrase in wordsets['pos']:
                if posphrase in tweet:
                    for negphrase in wordsets['neg']:
                        if negphrase in tweet:
                            predicted[-1] = "SARCASM"
        print(metrics.classification_report(test_set_y,predicted,target_names=categories))   

SVM with hinge loss
             precision    recall  f1-score   support

NOT_SARCASM       0.84      0.86      0.85      1511
    SARCASM       0.45      0.41      0.43       413

avg / total       0.76      0.77      0.76      1924

B
DatasetSarc
             precision    recall  f1-score   support

NOT_SARCASM       0.79      1.00      0.88      1511
    SARCASM       0.69      0.02      0.04       413

avg / total       0.77      0.79      0.70      1924

DatasetOther
             precision    recall  f1-score   support

NOT_SARCASM       0.79      1.00      0.88      1511
    SARCASM       0.50      0.01      0.01       413

avg / total       0.72      0.79      0.69      1924

A
DatasetSarc
             precision    recall  f1-score   support

NOT_SARCASM       0.79      1.00      0.88      1511
    SARCASM       0.75      0.01      0.01       413

avg / total       0.78      0.79      0.69      1924

DatasetOther
             precision    recall  f1-score   support

NOT_SARCASM 

In [65]:
c = defaultdict(int)
for label in test_set_y:
    c[label] += 1

In [66]:
c

defaultdict(int, {'NOT_SARCASM': 1511, 'SARCASM': 413})

In [67]:
413/1511

0.27332892124420916

In [74]:
pwd

'/home/ashish/Desktop/245/Project/code'